# 多項式環における計算

googleで検索して見つけた以下のページの行列の多項式への作用の定義を実装

https://ask.sagemath.org/question/60980/how-to-define-group-actions-on-sagemath/

In [1]:
from sage.categories.action import Action
from sage.groups.matrix_gps.matrix_group import is_MatrixGroup
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing

class InverseLinearMapSubstitutionAction(Action):
    def __init__(self, G, S):
        if not is_MatrixGroup(G):
            raise TypeError("Not a matrix group: %s" % G)
        if not is_MPolynomialRing(S):
            raise TypeError("Not a multivariate polynomial ring: %s" % S)
        if not G.degree() == S.ngens():
            raise ValueError("Degree of matrix group (%d) does not match number of generators of polynomial ring (%d)" % (G.degree(), S.ngens()))
        if not S.base_ring().has_coerce_map_from(G.base_ring()):
            raise ValueError("Base rings not compatible")
        super().__init__(G, S, is_left=True, op=operator.mul)

    def _act_(self, g, f):
        return f(*(g.inverse() * vector(self.domain(), self.domain().gens())))

    def _repr_name_(self):
        return "inverse linear map substitution action"

剰余類の代表系を計算する関数を実装

In [2]:
from sage.groups.group import is_Group
import time

def cosetRep(G,H):
    # 時間計測開始
    time_sta = time.perf_counter()
    if not is_Group(G):
        raise TypeError("Not a group: %s" % G)
    if not is_Group(H):
        raise TypeError("Not a group: %s" % H)
    
    rep=[]
    g=set(G.list())
    h=set(G(e) for e in H.list())
    coset=set()
    while g:
        p=g.pop()
        rep.append(p)
        coset = set(p*e for e in h)
        g=g-coset
        coset.clear()
        if len(rep)*H.order()==G.order():
            break;
            
    # 時間計測終了
    time_end = time.perf_counter()
    # 経過時間（秒）
    tim = time_end- time_sta

    print("%d 秒経過" % tim)
    #[結果]
    
    return rep

パラメタを設定

In [3]:
n=9
#r=2

A型のWeyl group $W_G$（の置換群表現版の$S_G$）を生成　※剰余類の代表系計算の軽量化のため、置換群を利用

In [4]:
Delta_G=RootSystem(['A',n-1])
S_G=WeylGroup(Delta_G,implementation="permutation")
DynkinDiagram(Delta_G)

O---O---O---O---O---O---O---O
1   2   3   4   5   6   7   8   
A8

cutoutするインデックスを設定

In [5]:
cutoutIndex=[3]
cutoutIndex.sort()

対応するcrossed Dynkin diagramを表示

In [6]:
CartanType(Delta_G).marked_nodes(cutoutIndex).dynkin_diagram()

O---O---X---O---O---O---O---O
1   2   3   4   5   6   7   8   
A8 with node 3 marked

simple reflectionを取得

In [7]:
s_G=S_G.simple_reflections()

uncrossed nodeに対応するsimple reflectionを取得

In [8]:
s_H=[ s_G[i] for i in set(range(1,len(s_G)+1))-set(cutoutIndex) ]

uncrossed nodeに対応するsimple reflectionから$W_G$の部分群$W_H$（の置換群表現版の$S_H$）を生成

In [9]:
S_H=S_G.subgroup(s_H)
print('W_Gの位数：%d'%S_G.order())
print('W_Hの位数：%d'%S_H.order())
print('代表系のサイズ：%d'%(S_G.order()/S_H.order()).round())

W_Gの位数：362880
W_Hの位数：4320
代表系のサイズ：84


冒頭で実装した関数を用いて, $W_G/W_H$（の置換群版の$S_G/S_H$）の代表系を計算

In [10]:
p_rep=cosetRep(S_G,S_H)

17 秒経過


作用させる際は行列としての作用となるため、本来ののWeyl group $W_G$を用意し, 代表系を$W_G$の元に変換

In [11]:
W_G=WeylGroup(Delta_G)
rep=[W_G(w) for w in p_rep]

多項式環とWeyl group $W_G$ の多項式環への作用を生成

In [12]:
R = PolynomialRing(QQ, 'x', W_G.degree())
a = InverseLinearMapSubstitutionAction(W_G, R); a

Left inverse linear map substitution action by Weyl Group of type ['A', 8] (as a matrix group acting on the ambient space) on Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6, x7, x8 over Rational Field

変数のリストを用意

In [13]:
x=R.gens();x

(x0, x1, x2, x3, x4, x5, x6, x7, x8)

positive root $P_G$をambient spaceに用意する.

In [14]:
#ambient spaceを生成
L=Delta_G.ambient_space()

#positive rootをambient spaceの元に変換
P_G=[ L(x) for x in Delta_G.root_lattice().positive_roots() ];P_G

[(1, -1, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, -1, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, -1, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, -1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, -1, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, -1, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, -1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, -1),
 (1, 0, -1, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, -1, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, -1, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, -1, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, -1, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, -1, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, -1),
 (1, 0, 0, -1, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, -1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, -1, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, -1, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, -1, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, -1),
 (1, 0, 0, 0, -1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, -1, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, -1, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, -1, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, -1),
 (1, 0, 0, 0, 0, -1, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, -1, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, -1, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, -1),
 (1, 0, 0, 0, 0, 0, -1, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, -1, 0),
 (0, 0, 

uncrossed nodeから生成されるpositive rootの集合$P_H$をambient spaceに用意する.

In [15]:
#root lattice内のsimple rootを取得
beta=Delta_G.root_lattice().simple_roots();beta

Finite family {1: alpha[1], 2: alpha[2], 3: alpha[3], 4: alpha[4], 5: alpha[5], 6: alpha[6], 7: alpha[7], 8: alpha[8]}

In [16]:
#positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
Pi_H=[ x for x in Delta_G.root_lattice().positive_roots()]
for i in cutoutIndex:
    Pi_H=[ x for x in Pi_H if not (x-beta[i]).is_positive_root()]
#ambient spaceの元に変換
P_H=[ L(x) for x in Pi_H ];P_H

[(1, -1, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, -1, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, -1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, -1, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, -1, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, -1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, -1),
 (1, 0, -1, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, -1, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, -1, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, -1, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, -1),
 (0, 0, 0, 1, 0, 0, -1, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, -1, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, -1),
 (0, 0, 0, 1, 0, 0, 0, -1, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, -1),
 (0, 0, 0, 1, 0, 0, 0, 0, -1)]

rootに対応する多項式を用意

In [17]:
Roots=set(P_G)-set(P_H)
alpha=[ -sum( r[l]*x[l] for l in range(len(x))) for r in Roots ];alpha

[-x1 + x6,
 -x0 + x6,
 -x0 + x5,
 -x1 + x5,
 -x2 + x3,
 -x1 + x4,
 -x1 + x3,
 -x0 + x3,
 -x0 + x4,
 -x0 + x7,
 -x1 + x8,
 -x2 + x5,
 -x2 + x8,
 -x0 + x8,
 -x2 + x6,
 -x2 + x4,
 -x2 + x7,
 -x1 + x7]

$ W_H $の作用で不変となる多項式を用意

In [18]:
f=(sum(x[l] for l in range(0,cutoutIndex[0])))^len(alpha)
factor(f)

(x0 + x1 + x2)^18

数値計算

乱数のリストを用意

In [19]:
y = [RealField(1000)(random()) for i in range(W_G.degree())];

In [20]:
v_y=[ rep[i].inverse()*vector(RealField(1000),y) for i in range(len(rep)) ]

In [21]:
sum(
    ((sum(z[l] for l in range(0,cutoutIndex[0])))^len(alpha))
    /prod([ -sum( r[l]*z[l] for l in range(len(z))) for r in Roots ])
    for z in  v_y  ).round()

87516

目的の計算を実施

In [22]:
sum( [ a(rep[i],f)/a( rep[i] , prod( alpha ) ) for i in range(len(rep)) ])

87516

--------以降は以前のノートの内容

パラメタ設定
（今はrは2で固定）

In [23]:
n=5
r=2

多項式環を用意

In [24]:
R0 = PolynomialRing(QQ, 'x', n)
R1= PolynomialRing(R0,'y',2)
phi = R1.flattening_morphism()
R = phi.codomain()

変数のリストを用意

In [25]:
x=R.gens();x

(x0, x1, x2, x3, x4, y0, y1)

$f=(y_0+y_1)^{r(n-r)}$を定義

In [26]:
f=(x[n]+x[n+1])^(r*(n-r))
factor(f)

(y0 + y1)^6

In [27]:
sum(sum( f.subs({x[n]:x[i],x[n+1]:x[j]})/prod([(x[k]-x[i])*(x[k]-x[j])
    for k in set(range(n))-{i,j}])
    for j in range(i+1,n))
    for i in range(0,n-1))

5

$\operatorname{deg} \operatorname{Gr}(2,n) = \frac{(2 n - 4)!}{(n - 2)! (n - 1)!}$と比較

In [28]:
factorial(2*n - 4)/(factorial(n - 2)*factorial(n - 1)) 

5

# 数値計算

パラメタの設定
（今は$r$は2で固定）

In [29]:
n=20
r=2

乱数の長さ$n$のリストを用意

In [30]:
x = [RealField(1000)(random()) for i in range(n)];

$f(y_0,y_1)=(y_0+y_1)^{r(n-r)}$を定義

In [31]:
y = SR.var('y', r); f(y0,y1)=(y0+y1)^(r*(n-r)); f

(y0, y1) |--> (y0 + y1)^36

In [32]:
(sum(sum(f(x[i],x[j])/prod([(x[k]-x[i])*(x[k]-x[j])
    for k in set(range(n))-{i,j}])
    for j in range(i+1,n))
    for i in range(0,n-1))
).round()

477638700

$\operatorname{deg} \operatorname{Gr}(2,n) = \frac{(2 n - 4)!}{(n - 2)! (n - 1)!}$と比較

In [33]:
 factorial(2*n - 4)/(factorial(n - 2)*factorial(n - 1)) 

477638700